In [2]:
import pandas as pd
import itertools
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
import nltk
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
import re
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer  
from nltk.corpus import words
from copy import deepcopy
from gensim.models import Word2Vec
from sklearn import svm


In [3]:
def preproc_text(text):
    text = re.sub(r"\d+", "", text.lower())
    for i in string.punctuation:
        text = text.replace(i,"")
    stop_words = set(stopwords.words("english"))
    tokens = word_tokenize(text)
    sample = [i for i in tokens if not i in stop_words]
    # lancaster_
    stemmer = LancasterStemmer() 
    lemmatizer = WordNetLemmatizer()

    
    
    return [lemmatizer.lemmatize(word) for word in sample]

In [4]:
data = pd.read_csv("movie_data.csv")
data.head()
review = data["review"] # create a data frame with only data 
review = review.apply(preproc_text)

In [5]:
model = Word2Vec(review[:40000], vector_size=500, window=10, min_count=10)

In [6]:
dic = set(model.wv.index_to_key)
def generate_vec(wordlist):
    feature = np.zeros(500)
    count = 0
    for word in wordlist:
        if word in dic:
            feature = np.add(model.wv[word], feature)
            count += 1
    
    return list(feature/count)
            

In [27]:
def split_train(original_train_data, size=10000):
    return original_train_data[:size], original_train_data[40000:40000+2000]

def binary_transform(sentiment):
    if sentiment == "positive":
        return 1
    return 0

In [37]:
label = data["sentiment"].apply(binary_transform)

In [38]:
train_X, val_X = split_train(review)
train_y, val_y = split_train(label)

In [39]:
train_X = train_X.apply(generate_vec)

In [40]:
X = np.zeros(10000*500).reshape(10000,500)
Y = np.array(train_y)
for i in range(5000):
    X[i] = train_X[i]

In [47]:
clf = svm.SVC()
clf.fit(X, Y)

SVC()

In [42]:
VX = val_X.apply(generate_vec).values
Vy = val_y.values

In [50]:
error = 0
for i in range(len(Vy)):
    if clf.predict([VX[i]]) != Vy[i]:
        error += 1
print("The accuracy is", (1 - error/len(Vy))*100, "%")

The accuracy is 85.6 %
